In [1]:
import os # импортируем библиотеку для работы с файловой системой

In [2]:
%pwd # проверяем текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../") # переходим в корневую директорию проекта

In [4]:
%pwd # проверяем текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project'

Update entity


In [5]:
from dataclasses import dataclass # импортируем декоратор для создания класса с полями-параметрами
from pathlib import Path # импортируем класс для работы с путями


@dataclass(frozen=True) # создаем класс с полями-параметрами
class DataTransformationConfig: # класс для хранения конфигурации
    root_dir: Path # путь к корневой директории проекта
    data_path: Path # путь к директории с исходными данными
    tokenizer_name: Path # путь к директории с токенизатором

Update the configuration manager in src config

In [6]:
from textSummarizer.constants import * # импортируем константы
from textSummarizer.utils.common import read_yaml, create_directories 
# импортируем функции для работы с конфигурацией и файловой системой

In [7]:
class ConfigurationManager: # класс для хранения конфигурации
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig: # функция для получения конфигурации
        config = self.config.data_transformation # получаем конфигурацию

        create_directories([config.root_dir]) # создаем директории

        data_transformation_config = DataTransformationConfig( 
            # создаем объект конфигурации
            root_dir=config.root_dir, 
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config # возвращаем конфигурацию

update the conponents

Токенизация — это процесс разделения текста на отдельные части, называемые токенами. Токены могут быть словами, числами, символами пунктуации или другими элементами текста.


In [9]:
import os # импортируем библиотеку для работы с файловой системой
from textSummarizer.logging import logger # импортируем логгер
from transformers import AutoTokenizer # импортируем токенизатор
from datasets import load_dataset, load_from_disk # импортируем функции для загрузки датасета

In [10]:
class DataTransformation: # класс для преобразования данных
    def __init__(self, config: DataTransformationConfig): # конструктор класса
        self.config = config # сохраняем конфигурацию
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) # загружаем токенизатор


    
    def convert_examples_to_features(self,example_batch): # функция для преобразования данных
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True ) 
        # токенизируем диалог
        
        with self.tokenizer.as_target_tokenizer(): # используем токенизатор для целевых данных
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True ) 
            # токенизируем суммарный текст
            
        return {
            'input_ids' : input_encodings['input_ids'], # возвращаем токенизированные данные
            'attention_mask': input_encodings['attention_mask'], # возвращаем маску внимания
            'labels': target_encodings['input_ids'] # возвращаем токенизированные целевые данные
        }
    

    def convert(self): # функция для преобразования данных
        dataset_samsum = load_from_disk(self.config.data_path) # загружаем датасет
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True) 
        # преобразуем данные
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset")) # сохраняем датасет

update the pipeline

In [11]:
try: # оборачиваем код в try-except для отлова ошибок
    config = ConfigurationManager() # создаем объект конфигурации
    data_transformation_config = config.get_data_transformation_config() # получаем конфигурацию
    data_transformation = DataTransformation(config=data_transformation_config) # создаем объект класса
    data_transformation.convert() # преобразуем данные
except Exception as e: # если произошла ошибка
    raise e # выбрасываем исключение

[2023-08-19 23:06:24,967: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-19 23:06:24,969: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-19 23:06:24,971: INFO: common: created directory at: artifacts]
[2023-08-19 23:06:24,973: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Mikhnin\miniconda3\envs\project_1\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 117172.93 examples/s]
